A* Search
This is a slightly modified version of the code I used in lab 3 it is mostly the same with a few bugfixes mostly here for comparative purposes

In [23]:
def searchAStar(graph,start,end,robust):
    unvisited = []
    current = None
    endpos = None
    count = 0
    for x in graph:
        if x[0] == end:
            endpos = x[5]
    for x in graph:
        x[4] = ((((x[5][0] - endpos[0]) ** 2) + ((x[5][1] - endpos[1]) ** 2) ** 2) ** .5)
    for x in graph:
        if x[0] == start:
            x[2] = 0
            current = x
            for child in x[1]:
                unvisited.append(child[0])
    while current[0] != end:
        count += 1
        if robust:
                print("Nodes Processed: " + str(count) + " Node ID: " + str(current[0])+ " Path Length: " + str(current[2]))
        if len(current[1]) == 1:
            check = None
            for x in unvisited:
                for node in graph:
                    if x == node[0]:
                        if check == None:
                            check = node
                        else:
                            if check[2] >  node [2]:
                                check = node
            current = check
            unvisited.remove(check[0])
        else:
            for child in current[1]:
                for node in graph:
                    if node[0] == child[0]:
                        if node[3] == None:
                            node[2] = child[1] + current[2]
                            node[3] = current[0]
                            unvisited.append(node[0])
                        elif node[2] > child[1] + current[2]:
                            node[2] = child[1] + current[2]
                            node[3] = current[0]
                            unvisited.append(node[0])
            check = None
            for x in unvisited:
                for node in graph:
                    if x == node[0]:
                        if check == None:
                            check = node
                        else:
                            if (check[2] + check[4])  >  (node[2] + node[4]):
                                check = node
            current = check
            unvisited.remove(check[0])

                
    for x in graph:
        if x[0] == end:
            path = []
            state = True
            z = x
            while state:
                
                for y in graph:
                    if y[0] == z[3]:
                        path.insert(0, y[0])
                        z = y
                    if z[3] == start:
                        path.append(end)
                        path.insert(0, start)
                        return [path, x[2] , count]

A* Landmark Search
My implementation of landmark A* the major differences are that it takes an additional peramiter in the form of the list of landmarks and then has to process heuristics for the landmarks and the end goal

Also added landmark handling which simply switches to the next set of heuristic valeus once a landmark is reached this also required that nodes store heuristic as a list of heuristics rather than a single value

In [25]:
def landmarkAStar(graph,start,end,robust,mark):
    unvisited = []
    current = None
    marks = []
    pos = []
    for x in mark:
        marks.append(x)
    marks.append(end)
    for x in marks:
        pos.append(None)
    count = 0
    land = 0
    cur = 0
    for y in marks:
        for x in graph:
            if x[0] == y:
                pos[cur] = x[5]
        cur += 1
    for x in graph:
        x[4] = []
    for x in graph:
        for y in pos:
            x[4].append(((((x[5][0] - y[0]) ** 2) + ((x[5][1] - y[1]) ** 2) ** 2) ** .5))
    for x in graph:
        if x[0] == start:
            x[2] = 0
            current = x
            for child in x[1]:
                unvisited.append(child[0])
    marpos = 0
    while current[0] != end:
        count += 1
        if robust:
                print("Nodes Processed: " + str(count) + " Node ID: " + str(current[0])+ " Path Length: " + str(current[2]))
        if len(current[1]) == 1:
            check = None
            for x in unvisited:
                for node in graph:
                    if x == node[0]:
                        if check == None:
                            check = node
                        else:
                            if check[2] >  node [2]:
                                check = node
            current = check
            unvisited.remove(check[0])
        else:
            for child in current[1]:
                for node in graph:
                    if node[0] == child[0]:
                        if node[3] == None:
                            node[2] = child[1] + current[2]
                            node[3] = current[0]
                            unvisited.append(node[0])
                        elif node[2] > child[1] + current[2]:
                            node[2] = child[1] + current[2]
                            node[3] = current[0]
                            unvisited.append(node[0])
            check = None
            for x in unvisited:
                for node in graph:
                    if x == node[0]:
                        if check == None:
                            check = node
                        else:
                            if (check[2] + check[4][marpos])  >  (node[2] + node[4][marpos]):
                                check = node
            current = check
            if current[0] == marks[marpos]:
                marpos += 1
                unvisited = []
                for x in current[1]:
                    if x[0] != current[3]:
                        unvisited.append(x[0])
            else:
                unvisited.remove(check[0])
            
                
    for x in graph:
        if x[0] == end:
            path = []
            state = True
            z = x
            while state:
                
                for y in graph:
                    if y[0] == z[3]:
                        path.insert(0, y[0])
                        z = y
                    if z[3] == start:
                        path.append(end)
                        path.insert(0, start)
                        return [path, x[2] , count]

Bidirectional Search
I have designed this algorithm to run two A* searches in conjunction one going backwards from the endpoint and one going foward from the start point one of the major differences is the addition of another piece of stored data in the node which allowed me to identify which algorithm had reached it and upon one algorithm finding a place the other algorithm had already visted it terminates the search and traces back both paths and provides the path straversed

In [40]:
def biderectional(graph,start,end,robust):
    unvisited = []
    unvisited1 = []
    current = None
    current1 = None
    endpos = None
    strpos = None
    count = 0
    ends = []
    for x in graph:
        if x[0] == end:
            endpos = x[5]
        if x[0] == start:
            strpos = x[5]
    for x in graph:
        x.append(None)
        x[4] = []
        x[4].append((((x[5][0] - endpos[0]) ** 2) + ((x[5][1] - endpos[1]) ** 2) ** 2) ** .5)
        x[4].append((((x[5][0] - strpos[0]) ** 2) + ((x[5][1] - strpos[1]) ** 2) ** 2) ** .5)
    for x in graph:
        if x[0] == start:
            x[2] = 0
            current = x
            for child in x[1]:
                unvisited.append(child[0])
        if x[0] == end:
            x[2] = 0
            current1 = x
            for child in x[1]:
                unvisited1.append(child[0])
    state = True
    while state:
        count += 1
        if robust:
                print("Nodes Processed: " + str(count * 2) + " Forward Node ID: " + str(current[0])+ " Backward Node ID: " + 
                    str(current1[0]) + " Path Lengths: " + str(current[2]) +" + "+ str(current1[2]) +" = "+str(current[2] + 
                    current1[2]))            
        if len(current[1]) == 1:
            check = None
            for x in unvisited:
                for node in graph:
                    if x == node[0]:
                        if check == None:
                            check = node
                        else:
                            if check[2] >  node [2]:
                                check = node
            current = check
            unvisited.remove(check[0])
        else:
            for child in current[1]:
                for node in graph:
                    if node[0] == child[0]:
                        if node[6] != None and node[6] != 'A':
                            ends.append(node)
                            ends.append(current1)
                            state = False
                        elif node[3] == None:
                            node[2] = child[1] + current[2]
                            node[3] = current[0]
                            node[6] = 'A'
                            unvisited.append(node[0])
                        elif node[2] > child[1] + current[2]:
                            node[2] = child[1] + current[2]
                            node[3] = current[0]
                            node[6] = 'A'
                            unvisited.append(node[0])
            if state:
                check = None
                for x in unvisited:
                    for node in graph:
                        if x == node[0]:
                            if check == None:
                                check = node
                            else:
                                if (check[2] + check[4][0])  >  (node[2] + node[4][0]):
                                    check = node
                current = check
                unvisited.remove(check[0])
        if state:
            if len(current1[1]) == 1:
                check = None
                for x in unvisited1:
                    for node in graph:
                        if x == node[0]:
                            if check == None:
                                check = node
                            else:
                                if check[2] >  node [2]:
                                    check = node
                current1 = check
                unvisited1.remove(check[0])
            else:
                for child in current1[1]:
                    for node in graph:
                        if node[0] == child[0]:
                            if node[6] != None and node[6] != 'B':
                                ends.append(node)
                                ends.append(current1)
                                state = False
                            elif node[3] == None:
                                node[2] = child[1] + current1[2]
                                node[3] = current1[0]
                                node[6] = 'B'
                                unvisited1.append(node[0])
                            elif node[2] > child[1] + current1[2]:
                                node[2] = child[1] + current1[2]
                                node[3] = current1[0]
                                node[6] = 'B'
                                unvisited1.append(node[0])
                if state:
                    check = None
                    for x in unvisited1:
                        for node in graph:
                            if x == node[0]:
                                if check == None:
                                    check = node
                                else:
                                    if (check[2] + check[4][1])  >  (node[2] + node[4][1]):
                                        check = node
                    current1 = check
                    unvisited1.remove(check[0])
    path1 = [] 
    path2 = []
    state = True
    z = ends[0]
    while z[0] != start:
        path1.append(z[0])
        temp = None
        for x in graph:
            if x[0] == z[3]:
                temp = x
                
        z = temp
        
    z = ends[1]
    while z[0] != end:
        path2.append(z[0])
        temp = None
        for x in graph:
            if x[0] == z[3]:
                temp = x
        z = temp
    path = path2
    for x in path1:
        path.insert(0,x)
    path.insert(0,start)
    path.append(end)
    return (path, ends[0][2] + ends[1][2] + 2, count)
            
    
          

                

Test Functions
Below are my test functions for each of the algorithms ran on identical data sets when ran they provide traces of how each algorithm went through the data set, the number of steps it took and the shortest path that was identified.

In [41]:
Robust = True
graph1 =[['A1', [['B1', 1], ['A2', 1]], None, None, None, [0, 0]], ['A2', [['A1', 1], ['A3', 1]], None, None, None, [0, 1]], ['A3', [['A2', 1], ['B3', 1]], None, None, None, [0, 2]], ['A4', [['B4', 1], ['A5', 1]], None, None, None, [0, 3]], ['A5', [['A4', 1], ['B5', 1]], None, None, None, [0, 4]], ['B1', [['A1', 1], ['C1', 1]], None, None, None, [1, 0]], ['B2', [['C2', 1]], None, None, None, [1, 1]], ['B3', [['A3', 1], ['B4', 1]], None, None, None, [1, 2]], ['B4', [['A4', 1], ['C4', 1], ['B3', 1]], None, None, None, [1, 3]], ['B5', [['A5', 1], ['C5', 1]], None, None, None, [1, 4]], 
         ['C1', [['B1', 1], ['C2', 1]], None, None, None, [2, 0]], ['C2', [['B2', 1], ['C1', 1]], None, None, None, [2, 1]], ['C3', [['D3', 1], ['C4', 1]], None, None, None, [2, 2]], ['C4', [['B4', 1], ['C3', 1]], None, None, None, [2, 3]], ['C5', [['B5', 1], ['D5', 1]], None, None, None, [2, 4]], ['D1', [['E1', 1], ['D2', 1]], None, None, None, [3, 0]], ['D2', [['D1', 1], ['E5', 1]], None, None, None, [3, 1]], ['D3', [['C3', 1]], None, None, None, [3, 2]], ['D4', [['E4', 1], ['D5', 1]], None, None, None, [3, 3]], ['D5', [['D4', 1], ['C5', 1]], None, None, None, [3, 4]], 
         ['E1', [['D1', 1], ['F1', 1]], None, None, None, [4, 0]], ['E2', [['D2', 1], ['E3', 1]], None, None, None, [4, 1]], ['E3', [['E2', 1], ['E4', 1], ['F3', 1]], None, None, None, [4, 2]], ['E4', [['E3', 1], ['E5', 1], ['D4', 1]], None, None, None, [4, 3]], ['E5', [['E4', 1], ['F5', 1]], None, None, None, [4, 4]], ['F1', [['E1', 1], ['F2', 1]], None, None, None, [5, 0]], ['F2', [['F1', 1], ['G2', 1]], None, None, None, [5, 1]], ['F3', [['E3', 1], ['F4', 1]], None, None, None, [5, 2]], ['F4', [['F3', 1], ['G4', 1]], None, None, None, [5, 3]], ['F5', [['E5', 1]], None, None, None, [5, 4]], 
         ['G1', [['G2', 1], ['H1', 1]], None, None, None, [6, 0]], ['G2', [['G1', 1], ['F2', 1], ['G3', 1]], None, None, None, [6, 1]], ['G3', [['G2', 1], ['H3', 1]], None, None, None, [6, 2]], ['G4', [['F4', 1]], None, None, None, [6, 3]], ['G5', [['H5', 1]], None, None, None, [6, 4]], ['H1', [['G1', 1], ['I1', 1]], None, None, None, [7, 0]], ['H2', [['H3', 1]], None, None, None, [7, 1]], ['H3', [['H2', 1], ['G3', 1]], None, None, None, [7, 2]], ['H4', [['I4', 1], ['H5', 1]], None, None, None, [7, 3]], ['H5', [['H4', 1], ['G5', 1], ['I5', 1]], None, None, None, [7, 4]],
         ['I1', [['H1', 1], ['J1', 1]], None, None, None, [8, 0]], ['I2', [['J2', 1], ['I3', 1]], None, None, None, [8, 1]], ['I3', [['I2', 1], ['I4', 1]], None, None, None, [8, 2]], ['I4', [['I3', 1], ['H4', 1], ['J4', 1]], None, None, None, [8, 3]], ['I5', [['H5', 1], ['J5', 1]], None, None, None, [8, 4]], ['J1', [['I1', 1], ['J2', 1]], None, None, None, [9, 0]], ['J2', [['J1', 1], ['I2', 1]], None, None, None, [9, 1]], ['J3', [['J4', 1]], None, None, None, [9, 2]], ['J4', [['J3', 1], ['I4', 1]], None, None, None, [9, 3]], ['J5', [['I5', 1]], None, None, None, [9, 4]]]
print(searchAStar(graph1,'A1','J5',Robust))
graph1 =[['A1', [['B1', 1], ['A2', 1]], None, None, None, [0, 0]], ['A2', [['A1', 1], ['A3', 1]], None, None, None, [0, 1]], ['A3', [['A2', 1], ['B3', 1]], None, None, None, [0, 2]], ['A4', [['B4', 1], ['A5', 1]], None, None, None, [0, 3]], ['A5', [['A4', 1], ['B5', 1]], None, None, None, [0, 4]], ['B1', [['A1', 1], ['C1', 1]], None, None, None, [1, 0]], ['B2', [['C2', 1]], None, None, None, [1, 1]], ['B3', [['A3', 1], ['B4', 1]], None, None, None, [1, 2]], ['B4', [['A4', 1], ['C4', 1], ['B3', 1]], None, None, None, [1, 3]], ['B5', [['A5', 1], ['C5', 1]], None, None, None, [1, 4]], 
         ['C1', [['B1', 1], ['C2', 1]], None, None, None, [2, 0]], ['C2', [['B2', 1], ['C1', 1]], None, None, None, [2, 1]], ['C3', [['D3', 1], ['C4', 1]], None, None, None, [2, 2]], ['C4', [['B4', 1], ['C3', 1]], None, None, None, [2, 3]], ['C5', [['B5', 1], ['D5', 1]], None, None, None, [2, 4]], ['D1', [['E1', 1], ['D2', 1]], None, None, None, [3, 0]], ['D2', [['D1', 1], ['E5', 1]], None, None, None, [3, 1]], ['D3', [['C3', 1]], None, None, None, [3, 2]], ['D4', [['E4', 1], ['D5', 1]], None, None, None, [3, 3]], ['D5', [['D4', 1], ['C5', 1]], None, None, None, [3, 4]], 
         ['E1', [['D1', 1], ['F1', 1]], None, None, None, [4, 0]], ['E2', [['D2', 1], ['E3', 1]], None, None, None, [4, 1]], ['E3', [['E2', 1], ['E4', 1], ['F3', 1]], None, None, None, [4, 2]], ['E4', [['E3', 1], ['E5', 1], ['D4', 1]], None, None, None, [4, 3]], ['E5', [['E4', 1], ['F5', 1]], None, None, None, [4, 4]], ['F1', [['E1', 1], ['F2', 1]], None, None, None, [5, 0]], ['F2', [['F1', 1], ['G2', 1]], None, None, None, [5, 1]], ['F3', [['E3', 1], ['F4', 1]], None, None, None, [5, 2]], ['F4', [['F3', 1], ['G4', 1]], None, None, None, [5, 3]], ['F5', [['E5', 1]], None, None, None, [5, 4]], 
         ['G1', [['G2', 1], ['H1', 1]], None, None, None, [6, 0]], ['G2', [['G1', 1], ['F2', 1], ['G3', 1]], None, None, None, [6, 1]], ['G3', [['G2', 1], ['H3', 1]], None, None, None, [6, 2]], ['G4', [['F4', 1]], None, None, None, [6, 3]], ['G5', [['H5', 1]], None, None, None, [6, 4]], ['H1', [['G1', 1], ['I1', 1]], None, None, None, [7, 0]], ['H2', [['H3', 1]], None, None, None, [7, 1]], ['H3', [['H2', 1], ['G3', 1]], None, None, None, [7, 2]], ['H4', [['I4', 1], ['H5', 1]], None, None, None, [7, 3]], ['H5', [['H4', 1], ['G5', 1], ['I5', 1]], None, None, None, [7, 4]],
         ['I1', [['H1', 1], ['J1', 1]], None, None, None, [8, 0]], ['I2', [['J2', 1], ['I3', 1]], None, None, None, [8, 1]], ['I3', [['I2', 1], ['I4', 1]], None, None, None, [8, 2]], ['I4', [['I3', 1], ['H4', 1], ['J4', 1]], None, None, None, [8, 3]], ['I5', [['H5', 1], ['J5', 1]], None, None, None, [8, 4]], ['J1', [['I1', 1], ['J2', 1]], None, None, None, [9, 0]], ['J2', [['J1', 1], ['I2', 1]], None, None, None, [9, 1]], ['J3', [['J4', 1]], None, None, None, [9, 2]], ['J4', [['J3', 1], ['I4', 1]], None, None, None, [9, 3]], ['J5', [['I5', 1]], None, None, None, [9, 4]]]
print(landmarkAStar(graph1,'A1','J5',Robust,['A5','E2','J1']))   
graph1 =[['A1', [['B1', 1], ['A2', 1]], None, None, None, [0, 0]], ['A2', [['A1', 1], ['A3', 1]], None, None, None, [0, 1]], ['A3', [['A2', 1], ['B3', 1]], None, None, None, [0, 2]], ['A4', [['B4', 1], ['A5', 1]], None, None, None, [0, 3]], ['A5', [['A4', 1], ['B5', 1]], None, None, None, [0, 4]], ['B1', [['A1', 1], ['C1', 1]], None, None, None, [1, 0]], ['B2', [['C2', 1]], None, None, None, [1, 1]], ['B3', [['A3', 1], ['B4', 1]], None, None, None, [1, 2]], ['B4', [['A4', 1], ['C4', 1], ['B3', 1]], None, None, None, [1, 3]], ['B5', [['A5', 1], ['C5', 1]], None, None, None, [1, 4]], 
         ['C1', [['B1', 1], ['C2', 1]], None, None, None, [2, 0]], ['C2', [['B2', 1], ['C1', 1]], None, None, None, [2, 1]], ['C3', [['D3', 1], ['C4', 1]], None, None, None, [2, 2]], ['C4', [['B4', 1], ['C3', 1]], None, None, None, [2, 3]], ['C5', [['B5', 1], ['D5', 1]], None, None, None, [2, 4]], ['D1', [['E1', 1], ['D2', 1]], None, None, None, [3, 0]], ['D2', [['D1', 1], ['E2', 1]], None, None, None, [3, 1]], ['D3', [['C3', 1]], None, None, None, [3, 2]], ['D4', [['E4', 1], ['D5', 1]], None, None, None, [3, 3]], ['D5', [['D4', 1], ['C5', 1]], None, None, None, [3, 4]], 
         ['E1', [['D1', 1], ['F1', 1]], None, None, None, [4, 0]], ['E2', [['D2', 1], ['E3', 1]], None, None, None, [4, 1]], ['E3', [['E2', 1], ['E4', 1], ['F3', 1]], None, None, None, [4, 2]], ['E4', [['E3', 1], ['E5', 1], ['D4', 1]], None, None, None, [4, 3]], ['E5', [['E4', 1], ['F5', 1]], None, None, None, [4, 4]], ['F1', [['E1', 1], ['F2', 1]], None, None, None, [5, 0]], ['F2', [['F1', 1], ['G2', 1]], None, None, None, [5, 1]], ['F3', [['E3', 1], ['F4', 1]], None, None, None, [5, 2]], ['F4', [['F3', 1], ['G4', 1]], None, None, None, [5, 3]], ['F5', [['E5', 1]], None, None, None, [5, 4]], 
         ['G1', [['G2', 1], ['H1', 1]], None, None, None, [6, 0]], ['G2', [['G1', 1], ['F2', 1], ['G3', 1]], None, None, None, [6, 1]], ['G3', [['G2', 1], ['H3', 1]], None, None, None, [6, 2]], ['G4', [['F4', 1]], None, None, None, [6, 3]], ['G5', [['H5', 1]], None, None, None, [6, 4]], ['H1', [['G1', 1], ['I1', 1]], None, None, None, [7, 0]], ['H2', [['H3', 1]], None, None, None, [7, 1]], ['H3', [['H2', 1], ['G3', 1]], None, None, None, [7, 2]], ['H4', [['I4', 1], ['H5', 1]], None, None, None, [7, 3]], ['H5', [['H4', 1], ['G5', 1], ['I5', 1]], None, None, None, [7, 4]],
         ['I1', [['H1', 1], ['J1', 1]], None, None, None, [8, 0]], ['I2', [['J2', 1], ['I3', 1]], None, None, None, [8, 1]], ['I3', [['I2', 1], ['I4', 1]], None, None, None, [8, 2]], ['I4', [['I3', 1], ['H4', 1], ['J4', 1]], None, None, None, [8, 3]], ['I5', [['H5', 1], ['J5', 1]], None, None, None, [8, 4]], ['J1', [['I1', 1], ['J2', 1]], None, None, None, [9, 0]], ['J2', [['J1', 1], ['I2', 1]], None, None, None, [9, 1]], ['J3', [['J4', 1]], None, None, None, [9, 2]], ['J4', [['J3', 1], ['I4', 1]], None, None, None, [9, 3]], ['J5', [['I5', 1]], None, None, None, [9, 4]]]
print(biderectional(graph1,'A1','I5',Robust))

Nodes Processed: 1 Node ID: A1 Path Length: 0
Nodes Processed: 2 Node ID: A2 Path Length: 1
Nodes Processed: 3 Node ID: A3 Path Length: 2
Nodes Processed: 4 Node ID: B3 Path Length: 3
Nodes Processed: 5 Node ID: B4 Path Length: 4
Nodes Processed: 6 Node ID: C4 Path Length: 5
Nodes Processed: 7 Node ID: A2 Path Length: 1
Nodes Processed: 8 Node ID: A4 Path Length: 5
Nodes Processed: 9 Node ID: C3 Path Length: 6
Nodes Processed: 10 Node ID: D3 Path Length: 7
Nodes Processed: 11 Node ID: B1 Path Length: 1
Nodes Processed: 12 Node ID: A5 Path Length: 6
Nodes Processed: 13 Node ID: B5 Path Length: 7
Nodes Processed: 14 Node ID: C5 Path Length: 8
Nodes Processed: 15 Node ID: D5 Path Length: 9
Nodes Processed: 16 Node ID: D4 Path Length: 10
Nodes Processed: 17 Node ID: E4 Path Length: 11
Nodes Processed: 18 Node ID: E5 Path Length: 12
Nodes Processed: 19 Node ID: F5 Path Length: 13
Nodes Processed: 20 Node ID: B1 Path Length: 1
Nodes Processed: 21 Node ID: E3 Path Length: 12
Nodes Processed: 